In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier

/content/drive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/classifier


In [6]:
pip install bresenham  #line drawing algorithm

In [7]:
import torch
import argparse

In [8]:
cd /content/drive/MyDrive/Univ of Surrey Work/sketch_adversarial-master

/content/drive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master


In [9]:
from models import *
from dataset import get_dataloader

ModuleNotFoundError: ignored

In [10]:
cd /content/drive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier

/content/drive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/classifier


In [11]:
from models import *
from dataset import get_dataloader

In [12]:
cd /content/drive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/rap_attack_sketch

/content/drive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master/rap_attack_sketch


In [13]:
ls

LICENSE       rap_attack.py         readme.md  target_model.py
__pycache__/  RAP_Run_Script.ipynb  records/


In [14]:
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import torch.nn.functional as F #233
import torch.optim as optim
from torchvision import datasets,models,transforms
from PIL import Image

In [15]:
from rap_attack import RAPAttack

In [18]:
import sys
sys.argv=['']
del sys

In [20]:
cd /content/drive/MyDrive/Univ of Surrey Work/sketch_adversarial-master

/content/drive/.shortcut-targets-by-id/19pG1VYZ1zMlJGiGZpQIbQ9Ub9qiy2CAG/Univ of Surrey Work/sketch_adversarial-master


In [21]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.RAPAttack_MNIST = {'num_classes': 10, 'overshoot': 0.02, 'max_iteration': 50}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    with torch.no_grad():
        True_Accuracy = 93.2250

    correct, correct_adv, correct_preserved = 0, 0, 0
    test_loss, test_loss_adv = 0, 0
    start_time = time.time()
 
    for i_batch, batch in enumerate(dataloader_Test):

        print(i_batch)
        images = batch['sketch_img'].to(device)
        images = (images > 0.4).float()

        for img, label in zip(images, batch['sketch_label']):
            target_label = torch.randint(0, 249, (1,))[0].to(device)
            img = img.unsqueeze(0)
            AdvExArray = stADV_attack_L_bfgs(img, target_label, model)
            # AdvExArray = AdvExArray.unsqueeze_(0).float()

            output = model(AdvExArray)
            test_loss_adv += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction_adv = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_adv += prediction_adv.eq(label.view_as(prediction_adv)).sum().item()

            AdvExArray_Bina =  (AdvExArray > 0.4).float()
            save_image(torch.cat((img, AdvExArray, AdvExArray_Bina), dim=0), 'images.jpg')
            # print(torch.equal(AdvExArray_Bina, img))
            output = model(AdvExArray_Bina)
            test_loss += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_preserved += prediction.eq(label.view_as(prediction)).sum().item()

    #        Save images


    Adv_Accuracy = 100. * correct_adv / len(dataloader_Test.dataset)
    Adv_Accuracy_preserved = 100. * correct_preserved / len(dataloader_Test.dataset)
    print('True_Accuracy: {:.4f}, Adv_Accuracy, Adv_Accuracy_preserved: {},  Time_Takes: {}\n'.format(True_Accuracy,
                                                                                                      Adv_Accuracy,
                                                                                                      Adv_Accuracy_preserved,
                                                                                                      (time.time() - start_time)))
        


FileNotFoundError: ignored

In [ ]:
if __name__ == '__main__':
    # Load the model
    model = MLP((1, 28, 28), 10)
    with open("records/mnist_model.pth", "rb") as f:
        model.load_state_dict(torch.load(f))
    model.eval()

    # Load the original example
    image = plt.imread("records/origin_example.jpg")[:, :, 0]
    image = image.reshape((1, 1, 28, 28))
    image = image / 255
    image = (image > 0.4)#thresholding pixel values
    # Set up the target and model wrapper
    target_label = 5
    smodel = SModel((1, 1, 28, 28), model, target = target_label)

    # Launch RAP attack
    adv, Sa = RAPAttack(image, smodel)
    adv = (adv > 0.4)#thresholding pixel values    
    # Display generated adversarial example and corresponding adversarial perturbations
    origin_prediction = model.predict(image).argmax(-1).item()
    adversarial_prediction = model.predict(adv).argmax(-1).item()

    perturbation = (adv - image)[0, 0]
    adv = adv[0,0]
    image = image[0, 0]

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('Prediction ' + str(origin_prediction))
    plt.subplot(1, 3, 2)
    plt.title('Perturbations')
    plt.imshow(perturbation, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1, 3, 3)
    plt.title('Adversarial Example')
    plt.imshow(adv, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('Prediction ' + str(adversarial_prediction))
    plt.savefig("records/rap_attack_result.jpg")
    plt.show()